# Technical background

The goal of this is to solve the following set of nonlinear coupled eigenequations which arise in the context of quantum electrodynamics, which studies the dynamics of electrons and electromagnetic field in which both entities are described quantum mechanically. The equations read:

\begin{align}
&H\psi + \alpha\sum\limits_{n=1}^{N_p}\sum\limits_{b=1}^{N_e}\frac{|\mathbf{F}_n(\mathbf{r}_0)|^2 p\psi_b\psi_b^{\dagger}p}{\omega_n(E_1-E_b-\omega_n)}\psi = E\psi  \\
& \text{det}\Big|I_3 + \lambda\mathbf{G}_n(\mathbf{r}_0,\mathbf{r}_0,\omega_n) = 0 \Big|,
\end{align}.

In this equation, the quantities that one wants to solve for are:

1. "matter" eigenvectors $\psi_i$ with $i=1\cdots N_e$ and the corresponding "matter" eigenvalues $E_1 \cdots E_{N_e}$.

2. "photon" eigenvectors $\mathbf{F}_n(\mathbf{r})$ with $n=1\cdots N_p$ and the corresponding "photon" eigenvalues $\omega_1 \cdots \omega_{N_p}$.

In this equation, $H$ and $p$  are known matrices, while $\alpha$ is a known constant, and $\lambda$ is a functional of all of the matter eigenvalues and eigenfunctions. $I_3$ is a $3\times 3$ identity matrix, and $\mathbf{G}$ is a Green's function, given in a spectral representation by
\begin{equation}
\mathbf{G}(\mathbf{r},\mathbf{r}',\omega) = \sum\limits_{k=1}^{N_p} \frac{\mathbf{F}_0(\mathbf{r})\mathbf{F}_0(\mathbf{r}')^{\dagger}}{\omega_{k0}^2-\omega^2},
\end{equation}
with $\mathbf{F}_{k0}$ and $\omega_{k0}$ being the photon eigenvectors and eigenvalues in the absence of matter ($\lambda = 0$). This is something that needs to be known by the user as it essentially specifies the optical system under consideration. 

Given the determination of the eigenvalues $\omega_n$, the corresponding eigenvectors are determined by:
\begin{equation}
\mathbf{F}_n(\mathbf{r}) = \lambda \mathbf{G}(\mathbf{r},\mathbf{r}_0,\omega_n)\mathbf{F}_n(\mathbf{r}_0).
\end{equation}

The first version of this code will solve these equations in one dimension.

## Equation-solving routines

This program makes substantial use of routines for solving nonlinear equations: both nonlinear equations in a single variable, and nonlinear eigenvalue problems.In order to run the routines in this code, you must first `Pkg.add("NLsolve")`.

- Nonlinear equations in a single variable are solved with Newton's method. This is the deault method. 

- Nonlinear eigenproblems are solved with fixed point iteration, which is achieved with `fixedpoint()`. The default method is `method = :anderson` with mixing parameter $m=0$ and damping parameter $\beta = 1$ (these parameters correspond to usual fixed-point iteration in which information about previous iterates is not explicitly used). A non-zero $m$ leads to a mixing of $m$ previous iterates in determining the next iterate.

Note that in some cases, we found that residuals of the first equation for the matter can vary wildly between iterations. In which case, consider using a non-zero $m$, as well as a damping parameter smaller than $1$.

For more information about the nonlinear routines, see: https://github.com/JuliaNLSolvers/NLsolve.jl

   

## User input and how to use this code 

To use this code, the user needs to define an electron and a photon, each of which have: 
- dimension of their Hilbert spaces
- eigenvalues corresponding to energies and frequencies
- eigenvectors corresponding to wavefunctions and electromagnetic modes

### Example of how to define an electron in a tight-binding or site-representation (note: a general real-space representation for the electron is a special case of this site-representation)

`electron_1d = construct_matter(zeros(N_el),0.25*ev_hartree,5,N_el)` constructs an electron with a 4-dimensional Hilbert space, coupled to its neighbors with tunneling energy 0.25*ev_hartree, with a "radius parameter" of 5, and with no on-site potential.

### Examples of how to define a photon 

#### A photon in a one-dimensional cavity

This can be done via the last line of the code, the function arguments being specified above.

`N_cav = 100;`

`d_cav = 1;`

`L_cav = 18904.0;` # 1 micron in atomic units

`S_cav = (10.0*nm)^2;` #area of cavity

`emitter_pos = L_cav/4;` # position where the emitter is located

`freqs_cav = pi*(1:N_cav)*c/L_cav;`

`modes_cav = zeros(d_cav,N_cav);`


`for counter = 1:N_cav`

`modes_cav[1,counter] = sqrt(2.0/L_cav/S_cav)*sin(freqs_cav[counter]*emitter_pos/c);` 

`end`

`pt_cav = photonic(N_cav,d_cav,freqs_cav,modes_cav)`

#### More general photon not given by a simple analytical treatment

If desired, the user can take results for modes and eigenvalues given by computational electromagnetism software such as MPB, MEEP, Lumerical, COMSOL, etc, and feed these into the photon class.

## Calculating photon eigenvalues due to coupling

This is achieved by the function `nlsolve(f_pt!,[guess])` which takes: a function that calculates the photon Green's function and its derivative at the location of the emitter. Note that the second argument is a guess for the frequency. This depends on your photon class, and the default suggestion is to search between every pair of modes by picking a guess in between. This is already done by the function `construct_coupled_freqs`

## Calculating electron eigenvalues due to coupling 

This is achieved by `fixedpoint(f_el2!, X0; method = :anderson, m=0,beta=1.0,iterations=200)` which does a fixed-point iteration to solve the matter equation for fixed photonic modes. To make this function work, you need to provide: an electron instance, a photon instance, a zero-matrix of dimension N_el by N_el (due to functions being written in-place), and a strength of the complicated term in the matter equation.

## Features to add in the next version

0. Make sure everything is converged. This is the number one priority.


1. Add self-consistent coupling between photon and electron eigenequations.


2. Add the possibility of having multiple electrons which are interacting, which will add a new nonlinear term to the first equation.


3. Make the outputs require less processing on the part of the user.